In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
from os import path
from pathlib import Path
import pandas as pd
import pickle
import seaborn as sns
import shutil
import time

from graphs.mixture_fit import best_fit_mixture
from models.gaussian_mixture import remove_outliers, gaussian_mixture
from preprocessing.exoplanets_gaia_crossmatch import gaia_exoplanets_cross, transform_to_cart
from preprocessing.download_gaia import GaiaDataset
from preprocessing.calc_density import get_densities

## Create folder structure

In [6]:
crossmatch_dir = "data/crossmatch/dr3"
#"~/OneDrive/phase_space_density_data/crossmatch/dr3" 

densities_dir = "data/densities/dr3"
#"~/OneDrive/phase_space_density_data/densities/dr3"

classification_dir = "data/classification/dr3"
#"~/OneDrive/phase_space_density_data/classification/dr3"

datasets_dir = "~/OneDrive/phase_space_density_data/initial_datasets"
#"data/initial_datasets"
#"~/OneDrive/phase_space_density_data/initial_datasets" 

# 1. Download Gaia dataset

In [5]:
GaiaDataset().get_gaia()

INFO: Login to gaia TAP server [astroquery.gaia.core]
OK
INFO: Login to gaia data server [astroquery.gaia.core]
OK
INFO: Query finished. [astroquery.utils.tap.core]


# 2. Crossmatch NASA Exoplanet dataset with Gaia dataset

In [4]:
def exoplanet_gaia_crossmatch(crossmatch_dir, transform_type="6d", table_name="gaiaedr3", save_spherical=True):
    """
    :param: transform_type: Type of coordinates transformation to perform on the data (6d, 5d_drop_vx, 5d_drop_vy or 5d_drop_vz).
    :param: table_name: Name of the Gaia dataset to use.
    :param: save_spherical: Save spherical values to a CSV file. When looping it is adviced to apply only once to save time. 
    
    :return: Density values for 1065 exoplanets and their neighbours, Winter-Gaia-NASA exoplanet archive crossmatch
    table containing 6D coordinates only and Winter-Gaia-NASA exoplanet archive crossmatch table with data from all 3
    sources combined.
    """

    # Cross match datasets and generate new ones.
    gaia = gaia_exoplanets_cross(f"{table_name}.csv", crossmatch_dir, save_gaia_id=True, return_data=True, save_spherical=save_spherical)
    transform_to_cart(gaia, table_name, crossmatch_dir, setting=transform_type)

In [5]:
exoplanet_gaia_crossmatch(crossmatch_dir, transform_type="6d", save_spherical=True)
exoplanet_gaia_crossmatch(crossmatch_dir, transform_type="5d_drop_vx", save_spherical=False)
exoplanet_gaia_crossmatch(crossmatch_dir, transform_type="5d_drop_vy", save_spherical=False)
exoplanet_gaia_crossmatch(crossmatch_dir, transform_type="5d_drop_rv", save_spherical=False)
exoplanet_gaia_crossmatch(crossmatch_dir, transform_type="5d_drop_vz", save_spherical=False)

KeyboardInterrupt: 

# 3. Calculate phase space density for neighbours of exoplanet hosts

In [7]:
def calculate_densities(star_labels_filename, dataset_filename, crossmatch_dir, densities_dir, exoplanets_only=True, start=0, stop=1, step=1, run_on_gpu=False): #[SR] swticthed stop=100000 to stop=1
    """
    Calculate phase space density for a given set of stars.
    
    :param: star_labels_filename: Name of the file containing star labels.
    :param: dataset_filename: Name of the file containing coordinates of the stars.
    :param: exoplanets_only: Compute density only for a list of exoplanets (~1000).
    :param: n_stars: Number of stars to calculate density for.
    :param: run_on_gpu: Use GPU accelerated pipeline.
    """
    #stop=100,000 will calc the phase-space densities of 100,000 stars
    #stop=100,000 can be changed. Can pick a star where you want to start and stop. This allows you to setect a random star to examine. Can start and a random interger and stop at +1
    
    labels_file = pd.read_csv(os.path.join(crossmatch_dir, star_labels_filename), dtype={"source_id": str, "Host": str})
    gaia = pd.read_csv(os.path.join(crossmatch_dir, dataset_filename))
                
    if gaia.shape[1] == 6: #Checks whether there are 6 columns, i.e. 6D information
        name = dataset_filename.split("_")[0] + f"_{dataset_filename[-6:-4]}" #what does the -6 and -4 do?
    else:
        name = dataset_filename.split("_")[0] + f"_{dataset_filename[-14:-4]}"

    if exoplanets_only:
        labels = labels_file["Host"].dropna() #why would the host be na?
        start = 0
        stop = labels.shape[0] #the stop point is the number of rows in labels
        step = 1
        name = name + "_only-" + dataset_filename.split("_")[1] + "s"
    else:
        labels = labels_file["source_id"]
        name = name + f"_{str(start)}_{str(stop)}"


    densities, dropped = get_densities(labels.to_numpy(), gaia.to_numpy(), start=start, stop=stop, step=step, run_on_gpu=run_on_gpu)
    
    with open(f"{densities_dir}/densities_{name}.data", "wb") as f:
        pickle.dump(densities, f)
    with open(f"{densities_dir}/dropped_densities_{name}.data", "wb") as f:
        pickle.dump(dropped, f)

In [8]:
for fname in os.listdir(crossmatch_dir):
    if "cartesian" in fname:
        if "5d_drop_rv" not in fname:
            continue
        print(fname)
        start = time.perf_counter()
        calculate_densities("gaiaedr3_star_labels.csv", fname, crossmatch_dir, densities_dir, exoplanets_only=True, run_on_gpu=False) #[SR] set run_on_gpu=False
        end = time.perf_counter()
        print(f"{fname} completed in: {str(end-start)}")

        start = time.perf_counter()
        calculate_densities("gaiaedr3_star_labels.csv", fname, crossmatch_dir, densities_dir, exoplanets_only=False, run_on_gpu=False) #[SR] set run_on_gpu=False
        end = time.perf_counter()
        print(f"{fname} completed in: {str(end-start)}")



gaiaedr3_exoplanet_cross_cartesian_5d_drop_rv.csv


KeyboardInterrupt: 

# 3. Fit gaussian mixture model to predict if target star belongs to overdensity or underdensity group. Return scores and attributes of the model.

In [ ]:
def labels(row):
    if row["gm_p_low"] >= 0.84:
        return "0"
    elif row["gm_p_high"] >= 0.84:
        return "2"
    else:
        return "1"

In [ ]:
def fit_gaussian_mixture(file_name, classification_dir, densities_dir, show_graph=True, save_graph=True): # [SR] show_graph=False, save_graph=False switched to show_graph=True, save_graph=True

    sigma = 2
    fig_dir=None
    file_name = file_name.split(".")[0]
    if save_graph:  
        if os.path.isdir(f"figures/{file_name}"):
            shutil.rmtree(f"figures/{file_name}")
        fig_dir = file_name
    
        os.mkdir(f"figures/{fig_dir}")
    
    with open(f"{densities_dir}/{file_name}.data", "rb") as f:
        densities = pickle.load(f)

    results = []
    for i in densities:
        # Compute log10 of the host density and expand dimensions for further use
        target = np.expand_dims(np.log10(i[1]), axis=0).T

        # Remove outliers outside sigma
        data = remove_outliers(i[4], sigma=sigma)
        
        # Apply gaussian mixture model to the data
        model, scores = gaussian_mixture(data, [target], components=2, scores_only=False)

        # Create list consisting of star name and its density for graph drawing
        scores.insert(0, target[0])
        scores.insert(0, np.log10(i[4].min()))
        scores.insert(0, np.log10(i[4].max()))
        scores.insert(0, np.log10(i[4].std()))
        scores.insert(0, np.log10(i[4].mean()))
        scores.insert(0, i[3])
        scores.insert(0, i[2])
        scores.insert(0, i[0])
        
        results.append(scores)
        
        # Draw best fit mixture
        if type(i[0]) != str:
            host = [f"{i[0]:.0f}", target]
        else:
            host = [i[0], target]
        
        if show_graph or save_graph:
            best_fit_mixture(model, data, host, results[densities.index(i)][4], fig_dir, show_graph, save_graph)

    
    df = pd.DataFrame(results, columns=["Host", "n_40pc_stars", "n_80pc_stars", "densities_mean", "densities_std", 
                                        "densities_max", "densities_min", "target_density", "gm_p_low", "gm_p_high",
                                        "gm_mean_low", "gm_mean_high", "gm_cov_low", "gm_cov_high", "gm_aic", "gm_bic"])
    df["class"] = df.apply(lambda row: labels(row), axis=1)

    df.to_csv(f"{classification_dir}/features_{file_name}.csv")

In [ ]:
features = os.listdir(classification_dir)
for fname in os.listdir(densities_dir):
    for i in features:
        if fname.split(".")[0] in i:
            continue
    fit_gaussian_mixture(fname, classification_dir, densities_dir, show_graph=False, save_graph=False)